<a href="https://colab.research.google.com/github/wyattowalsh/sports-analytics/blob/main/basketball/notebooks/data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align='center'> Data Collection for the <i><b>Basketball Dataset</b></i> </h1>

This notebook contains the associated work necessary to collect the data that composes the [***Kaggle Basketball Dataset*** (wyattowalsh/basketball)](https://www.kaggle.com/wyattowalsh/basketball) and serves as the foundation for the [basketball related projects](https://github.com/wyattowalsh/sports-analytics/tree/main/basketball) within my [sports analytics GitHub repository](https://github.com/wyattowalsh/sports-analytics).

One of the goals for the data collection component of this project is to produce a `robust`, *organized* dataset that can grow to as **large of a scale** as possible. You can find an explanation of my solution for storing the files related to the [***Basketball Dataset***](https://www.kaggle.com/wyattowalsh/basketball) below.

<img src="https://unsplash.com/photos/Kv-gAzpUSRg/download?force=true">

## Overview

***Kaggle*** offers many formats of which one can save files to a dataset, which include: `CSV`, `JSON`, `SQLite`, and `Archives`, among others. The platform essentially acts similarly to industrial cloud solutions like *Google Cloud Platform's* (**GCP**) ***Cloud Storage*** or *Amazon Web Service's* (**AWS**) ***S3*** albeit with a **100GB** storage capacity. ***Kaggle*** datasets as well as these industrial solutions can be considered as broad object/file storage and in certain data engineering paradigms can serve as data lakes. 

It seems that many state-of-the-art (SOTA) data storage solutions pivot around an organizational-wide data lake (of which itself allows for general object storage) that has multiple inputs (*"tributaries"*) both streaming into and routinely added to the overall lake. One benefit of this paradigm is that the lake facilitates the storage of both structured (tabular) and unstructured (image, video, audio, text, etc) data. This can prove useful because, as time progresses, new techniques for extracting useful information from unstructured data can be utilized. Thus it also seems like a good idea to hold onto all extracted data, if possible. 

***Kaggle*** datasets can serve as data lakes through the archival process or simply by storing data files in their raw file format. This certainly serves as a strong foundation for building a &#8212; one day in the future &#8212; <b><i>"big data"</i></b> collection. 

However, there is further work that can be done in configuring ***Kaggle*** datasets to enable additional platform functionality as well as improved storage efficiency. Structured data, whether structured upon extraction or structured through some pre-processing, can be stored in a ***SQLite*** database (`.sqlite` file type) as opposed to storing individual files such as `CSVs` or `JSONs` within the dataset. Thus, a single database file is stored as an object within the dataset, enabling additional functionality. One easily discerned advantage with storing in ***SQLite*** is that histograms of the distribution of across continuous variables are given directly within ***Kaggle***. 

As this project moves forward, I hope to collect a large collection of both structured and unstructured data. I hope that the ***SQLite*** database (`basketball.sqlite`) can serve to house the structured data in an efficient, useful format, similarly to the [***European Soccer Database***](https://www.kaggle.com/hugomathien/soccer).

## View System Information

In [ ]:
print("********************** CUDA Version ********************** \n - \n")
!nvcc --version
print("********************** CPU Info ********************** \n - \n")
!cat /proc/cpuinfo
print("********************** CPU Count ********************** \n - \n")
import os
print(os.cpu_count())
print("********************** GPU Info ********************** \n - \n")
!nvidia-smi
print("********************** Python Version ********************** \n - \n")
!python -V

## Prepare Development Environment

### Clone Project Repository and Install Dependencies

In [ ]:
# remove sample data and clone repo
!rm -r sample_data/
!rm -r sports-analytics/
!git clone https://github.com/wyattowalsh/sports-analytics.git

# change directory to directory that contains this notebook
%cd /content/sports-analytics/basketball/notebooks/

# install dependencies
!pip install -r ../../dependencies/basketball/data_collection.txt

### Import Dependencies and Enable Tools

In [29]:
# nba_api dependencies
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import commonplayerinfo, playercareerstats, teamdetails, leaguegamelog, boxscoresummaryv2,\
                                    alltimeleadersgrids

# datascience stack
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import sqlite3 as sql

# system utility stack
import os
import time
import urllib
from functools import partial

pd.options.display.max_columns = None

# # Upload kaggle.json to /content/
# from google.colab import files 
# uploaded = files.upload()

# Move and change permissions as needed, allowing for import
# !mkdir -p ~/.kaggle/ && cp kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
import kaggle

# # change directory to directory that contains this notebook
# %cd /content/sports-analytics/basketball/notebooks/

# # utilize Colab Monitor
# from urllib.request import urlopen
# exec(urlopen("http://colab-monitor.smankusors.com/track.py").read())
# _colabMonitor = ColabMonitor().start()

## Collect Data

### Connect to Database

In [17]:
conn = sql.connect('../data/basketball.sqlite')

### Players

#### Get Players DataFrame and Type ID as String

In [ ]:
df_players = pd.DataFrame(players.get_players()).astype({'id': 'str'})
df_players

In [ ]:
df_players.info()

#### Add Dataframe as Table to Database, Unless it Already Exists

In [ ]:
try:
    df_players.to_sql('Player', conn, index=False)
except:
    pass

### Teams

#### Get Teams DataFrame, Type ID as String and Convert Year to Datetime

In [ ]:
df_teams = pd.DataFrame(teams.get_teams()).astype({'id': 'str'})
df_teams['year_founded'] =  pd.to_datetime(df_teams['year_founded'], format='%Y').dt.year # convert year to datetime type
df_teams.head()

In [ ]:
df_teams.info()

#### Add Dataframe as Table to Database, Unless it Already Exists

In [ ]:
try:
    df_teams.to_sql('Team', conn, index=False)
except:
    pass

### Common Player Information

In [ ]:
# define function to extract common player info for a single player
def get_common_player_info(player_id, proxies):
    # define helpful variables
    no_res = True
    proxy_collection_counter = 0
    proxy_index = 0
    # while no response
    while no_res:
        # try getting a response without a proxy
        try:
            res = commonplayerinfo.CommonPlayerInfo(player_id=player_id, timeout=3).get_data_frames()
            no_res = False
            print(player_id)
            break
        except:
            # if that fails
            while no_res:
                # try getting with a certain proxy
                try: 
                    res = commonplayerinfo.CommonPlayerInfo(player_id=player_id, proxy="http://" + proxies[proxy_index], timeout=3).get_data_frames()
                    no_res = False
                    break
                except:
                    # if that fails, move on to next proxy unless out of proxies
                    if (proxy_index + 1) >= len(proxies):
                        # unless tried proxies 5 times
                        if proxy_collection_counter < 6:
                            # if out of proxies: get more proxies, fix counters, and try without a proxy again
                            proxy_index = 0
                            proxy_collection_counter = proxy_collection_counter + 1
                            print(player_id, ' failed {} times'.format(proxy_collection_counter))
                            proxies = [str(proxy).split('\\')[0][2:] for proxy in urllib.request.urlopen("https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=1000&country=all&ssl=yes&anonymity=all&simplified=true").readlines()]
                            break
                        else:
                            return None
                    else:
                        proxy_index = proxy_index + 1
                        
    # merge the common player info and player headline stats and drop timeframe                   
    res_df = pd.merge(res[0], res[1], how='left', left_on=['PERSON_ID', 'DISPLAY_FIRST_LAST'], right_on=['PLAYER_ID', 'PLAYER_NAME'])
    res_df = res_df.drop(['TimeFrame'], axis=1)
    return res_df

# get proxies
proxies = [str(proxy).split('\\')[0][2:] for proxy in urllib.request.urlopen("https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=1000&country=all&ssl=yes&anonymity=all&simplified=true").readlines()]

# get common player info for each player in the db
dfs = []
player_ids = pd.read_sql('SELECT id FROM Player', conn)['id'].values
dfs = [get_common_player_info(player_id=player_id, proxies=proxies) for player_id in player_ids]
df = pd.concat(dfs)
df.head()

In [ ]:
dff = df.copy()
df.rename(columns = {'PERSON_ID':'ID'}, inplace = True)
df['ID'] = df["ID"].astype(str)
df['BIRTHDATE'] = df['BIRTHDATE'].astype(str)
df['HEIGHT'] = df['HEIGHT'].apply(lambda x: int(x.split('-')[0])*12 + int(x.split('-')[1]) \
                                  if len(x.strip()) > 0 else np.nan)
df['WEIGHT'] = df['WEIGHT'].apply(lambda x: int(x) if len(x.strip()) > 0 else np.nan)
df['TEAM_ID'] = df['TEAM_ID'].astype(str)
df['FROM_YEAR'] = df['FROM_YEAR'].astype(str)
df['TO_YEAR'] = df['TO_YEAR'].astype(str)
df['DRAFT_YEAR'] = df['DRAFT_YEAR'].astype(str)
df['DRAFT_ROUND'] = df['DRAFT_ROUND'].astype(str)
df['DRAFT_NUMBER'] = df['DRAFT_NUMBER'].astype(str)
df = df.drop(['PLAYER_ID', 'PLAYER_NAME'], axis=1)
df['PTS'] = pd.to_numeric(df['PTS'], errors='coerce')
df['AST'] = pd.to_numeric(df['AST'], errors='coerce')
df['REB'] = pd.to_numeric(df['REB'], errors='coerce')
df['ALL_STAR_APPEARANCES'] = pd.to_numeric(df['ALL_STAR_APPEARANCES'], errors='coerce')
df['PIE'] = pd.to_numeric(df['PIE'], errors='coerce')
df = df.reset_index(drop=True)

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.to_sql('Player_Attributes', conn, index=False, if_exists='replace')

In [ ]:
try:
    df.to_sql('Player_Attributes', conn, index=False)
except:
    pass

#### Upload to Kaggle

In [ ]:
!kaggle datasets version -p ../data -m "adding common player info"

### Team Details

In [ ]:
# define function to extract common player info for a single player
def get_team_details(team_id, proxies):
    # define helpful variables
    no_res = True
    proxy_collection_counter = 0
    proxy_index = 0
    # while no response
    while no_res:
        # try getting a response without a proxy
        try:
            res = teamdetails.TeamDetails(team_id, timeout=3).get_data_frames()
            no_res = False
            print(team_id)
            break
        except:
            # if that fails
            while no_res:
                # try getting with a certain proxy
                try: 
                    res = teamdetails.TeamDetails(team_id, proxy="http://" + proxies[proxy_index], timeout=3).get_data_frames()
                    no_res = False
                    break
                except:
                    # if that fails, move on to next proxy unless out of proxies
                    if (proxy_index + 1) >= len(proxies):
                        # unless tried proxies 5 times
                        if proxy_collection_counter < 5:
                            # if out of proxies: get more proxies, fix counters, and try without a proxy again
                            proxy_index = 0
                            proxy_collection_counter = proxy_collection_counter + 1
                            print(team_id, ' failed {} times'.format(proxy_collection_counter))
                            proxies = [str(proxy).split('\\')[0][2:] for proxy in urllib.request.urlopen("https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=1000&country=all&ssl=yes&anonymity=all&simplified=true").readlines()]
                            break
                        else:
                            return None
                    else:
                        proxy_index = proxy_index + 1
                        
    # merge the common player info and player headline stats and drop timeframe  
    dfs = res
    df = dfs[0]
    try:
        df['FACEBOOK_WEBSITE_LINK'] = dfs[2].loc[dfs[2]['ACCOUNTTYPE'] == 'Facebook']['WEBSITE_LINK'].values[0]
    except:
        df['FACEBOOK_WEBSITE_LINK'] = np.nan
    try:
        df['INSTAGRAM_WEBSITE_LINK'] = dfs[2].loc[dfs[2]['ACCOUNTTYPE'] == 'Instagram']['WEBSITE_LINK'].values[0]
    except:
        df['INSTAGRAM_WEBSITE_LINK'] = np.nan
    try:
        df['TWITTER_WEBSITE_LINK'] = dfs[2].loc[dfs[2]['ACCOUNTTYPE'] == 'Twitter']['WEBSITE_LINK'].values[0]
    except:
        df['TWITTER_WEBSITE_LINK'] = np.nan
    df.rename(columns = {'TEAM_ID':'ID'}, inplace = True)
    df['ID'] = df["ID"].astype(str)
    df['YEARFOUNDED'] = df['YEARFOUNDED'].astype(str)
    df['ARENACAPACITY'] = pd.to_numeric(df['ARENACAPACITY'], errors='coerce')
    df_1 = dfs[1]
    df_1.rename(columns = {'TEAM_ID':'ID'}, inplace = True)
    df_1['ID'] = df_1["ID"].astype(str)
    df_1['YEARFOUNDED'] = df_1['YEARFOUNDED'].astype(str)
    df_1['YEARACTIVETILL'] = df_1['YEARACTIVETILL'].astype(str)
    return [df, df_1]

# get proxies
proxies = [str(proxy).split('\\')[0][2:] for proxy in urllib.request.urlopen("https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=1000&country=all&ssl=yes&anonymity=all&simplified=true").readlines()]

# get common player info for each player in the db
dfs = []
team_ids = pd.read_sql('SELECT id FROM Team', conn)['id'].values
dfs = [get_team_details(team_id, proxies=proxies) for team_id in team_ids]
df_0 = pd.concat([df[0] for df in dfs])
df_1 = pd.concat([df[1] for df in dfs])
display(df_0.head())
df_1.head()

In [ ]:
df_0.info()

In [ ]:
df_1.info()

In [ ]:
try:
    df_0.to_sql('Team_Attributes', conn, index=False)
except:
    pass
try:
    df_1.to_sql('t', conn, index=False)
except:
    pass

#### Upload to Kaggle

In [ ]:
!kaggle datasets version -p ../data -m "adding team details"

### League Games

In [ ]:
def get_league_games(season_id, proxies):
    # define helpful variables
    no_res = True
    proxy_collection_counter = 0
    proxy_index = 0
    # while no response
    while no_res:
        # try getting a response without a proxy
        try:
            res = leaguegamelog.LeagueGameLog(season=season_id, timeout=5).get_data_frames()
            no_res = False
            print(season_id)
            break
        except:
            # if that fails
            while no_res:
                # try getting with a certain proxy
                try: 
                    res = leaguegamelog.LeagueGameLog(season=season_id, proxy="http://" + proxies[proxy_index], timeout=5).get_data_frames()
                    no_res = False
                    break
                except:
                    # if that fails, move on to next proxy unless out of proxies
                    if (proxy_index + 1) >= len(proxies):
                        # unless tried proxies 5 times
                        if proxy_collection_counter < 5:
                            # if out of proxies: get more proxies, fix counters, and try without a proxy again
                            proxy_index = 0
                            proxy_collection_counter = proxy_collection_counter + 1
                            print(season_id, ' failed {} times'.format(proxy_collection_counter))
                            proxies = [str(proxy).split('\\')[0][2:] for proxy in urllib.request.urlopen("https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=1500&country=all&ssl=yes&anonymity=all&simplified=true").readlines()]
                            break
                        else:
                            return None
                    else:
                        proxy_index = proxy_index + 1
                        
  
    df = res[0]
    df["TEAM_ID"] = df["TEAM_ID"].astype(str)
    game_ids = df["GAME_ID"].unique()         

    def get_df(df, game_id):
        season_id = df['SEASON_ID'].values[0]
        game_date = df['GAME_DATE'].values[0]
        rows = df.loc[df["GAME_ID"] == game_id].drop(['GAME_ID', "SEASON_ID", "GAME_DATE"], axis=1)
        row_0 = rows.iloc[[0]]
        row_1 = rows.iloc[[1]]

        def rename_cols(df):
            if "vs" in df['MATCHUP'].values[0]:
                df.columns = [x + '_HOME' for x in df.columns]
            else:
                df.columns = [x + '_AWAY' for x in df.columns]
            return df
        
        row_0 = rename_cols(row_0).reset_index(drop=True)
        row_1 = rename_cols(row_1).reset_index(drop=True)
        df = pd.concat([row_0, row_1], axis=1)
        cols = list(df.columns.values)
        cols = ['GAME_ID', "SEASON_ID"] + cols
        df["GAME_ID"] = game_id
        df["SEASON_ID"] = season_id
        df = df[cols]
        return df
    
    df = pd.concat([get_df(df, game_id) for game_id in game_ids], axis=0)
    return df

# get proxies
proxies = [str(proxy).split('\\')[0][2:] for proxy in urllib.request.urlopen("https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=1500&country=all&ssl=yes&anonymity=all&simplified=true").readlines()]

# get common player info for each player in the db
dfs = []
player_ids = pd.read_sql('SELECT id FROM Player', conn)['id'].values
season_ids = [str(x) + "-" + str(x+1)[2:4] for x in range(1946, 2021)]
dfs = [get_league_games(season_id=season_id , proxies=proxies) for season_id in season_ids]
df = pd.concat(dfs)
df.head()

In [ ]:
df

In [ ]:
df.info()

In [ ]:
try:
    df.to_sql('Game', conn, index=False)
except:
    pass

#### Upload to Kaggle

In [ ]:
!kaggle datasets version -p ../data -m "adding league game log to Game table"

### Box Score Summaries

In [ ]:
!kaggle datasets init

In [ ]:
def get_box_score_summaries(game_id, proxies):
    # define helpful variables
    no_res = True
    proxy_collection_counter = 0
    proxy_index = 0
    # while no response
    while no_res:
        # try getting a response without a proxy
        try:
            res = boxscoresummaryv2.BoxScoreSummaryV2(game_id, timeout=5).get_data_frames()
            no_res = False
            break
        except:
            # if that fails
            while no_res:
                # try getting with a certain proxy
                try: 
                    res = boxscoresummaryv2.BoxScoreSummaryV2(game_id, proxy="http://" + proxies[proxy_index], timeout=5).get_data_frames()
                    no_res = False
                    break
                except:
                    # if that fails, move on to next proxy unless out of proxies
                    if (proxy_index + 1) >= len(proxies):
                        # unless tried proxies 5 times
                        if proxy_collection_counter < 5:
                            # if out of proxies: get more proxies, fix counters, and try without a proxy again
                            proxy_index = 0
                            proxy_collection_counter = proxy_collection_counter + 1
                            print(game_id, ' failed {} times'.format(proxy_collection_counter))
                            proxies = [str(proxy).split('\\')[0][2:] for proxy in urllib.request.urlopen("https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=1500&country=all&ssl=yes&anonymity=all&simplified=true").readlines()]
                            break
                        else:
                            return None
                    else:
                        proxy_index = proxy_index + 1
                        
  
    game_summary = res[0]
    home_team_id = game_summary['HOME_TEAM_ID'].values[0]
    visitor_team_id = game_summary['VISITOR_TEAM_ID'].values[0]
    game_summary['GAME_STATUS_ID'] = game_summary['GAME_STATUS_ID'].astype(str)
    game_summary['HOME_TEAM_ID'] = game_summary['HOME_TEAM_ID'].astype(str)
    game_summary['VISITOR_TEAM_ID'] = game_summary['VISITOR_TEAM_ID'].astype(str)
    
    other_stats = res[1]
    other_stats['TEAM_ID'] = other_stats['TEAM_ID'].astype(str)
    other_stats_home = other_stats.loc[other_stats['TEAM_ID'] == str(home_team_id)].drop(['LEAGUE_ID'], axis=1).reset_index(drop=True)
    other_stats_visitor = other_stats.loc[other_stats['TEAM_ID'] == str(visitor_team_id)].drop(['LEAGUE_ID'], axis=1).reset_index(drop=True)
    try:
        league_id = other_stats['LEAGUE_ID'].values[0]
    except:
        try: 
            league_id = other_stats['LEAGUE_ID'].values
        except:
            league_id = None

    other_stats_home.columns = [col + '_HOME' for col in other_stats_home.columns]
    other_stats_visitor.columns = [col + '_AWAY' for col in other_stats_visitor.columns]
    other_stats = pd.concat([other_stats_home, other_stats_visitor], axis = 1)
    other_stats['LEAGUE_ID'] = league_id
    
    df = pd.concat([game_summary, other_stats], axis=1)
    
    officials = res[2]
    officials["GAME_ID"] = game_id
    officials['OFFICIAL_ID'] = officials['OFFICIAL_ID'].astype(str)
    
    inactive_players = res[3]
    inactive_players["GAME_ID"] = game_id
    inactive_players["PLAYER_ID"] = inactive_players["PLAYER_ID"].astype(str)
    inactive_players["TEAM_ID"] = inactive_players["TEAM_ID"].astype(str)
    
    
    
    game_info = res[4]
    
    df = pd.concat([df, game_info], axis=1)
    
    line_score = res[5]
    line_score['TEAM_ID'] = line_score['TEAM_ID'].astype(str)
    line_score = line_score.drop(['GAME_DATE_EST', "GAME_SEQUENCE", "GAME_ID"], axis=1)
    line_score_home = line_score.loc[line_score['TEAM_ID'] == str(home_team_id)].reset_index(drop=True)
    line_score_visitor = line_score.loc[line_score['TEAM_ID'] == str(visitor_team_id)].reset_index(drop=True)
    line_score_home.columns = [col + '_HOME' for col in line_score.columns]
    line_score_visitor.columns = [col + '_AWAY' for col in line_score.columns]
    line_score = pd.concat([line_score_home, line_score_visitor], axis = 1)
   
    df = pd.concat([df, line_score], axis=1)
    
    
    last_meeting = res[6]
    last_meeting = last_meeting.drop(['GAME_ID'], axis=1)
    last_meeting['LAST_GAME_HOME_TEAM_ID'] = last_meeting['LAST_GAME_HOME_TEAM_ID'].astype(str)
    last_meeting['LAST_GAME_VISITOR_TEAM_ID'] = last_meeting['LAST_GAME_VISITOR_TEAM_ID'].astype(str)
    
    df = pd.concat([df, last_meeting], axis=1)
    
    season_series = res[7]
    season_series = season_series.drop(['GAME_ID', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'GAME_DATE_EST'], axis=1)
    season_series = season_series.rename({'LAST_GAME_VISITOR_TEAM_CITY1': "LAST_GAME_VISITOR_TEAM_CITY"}, axis=1)
    
    df = pd.concat([df, season_series], axis=1)
    
    available_video = res[8]
    available_video = available_video.drop(['GAME_ID'], axis=1)
    
    df = pd.concat([df, available_video], axis=1).drop(['TEAM_ID_HOME', 'TEAM_ID_AWAY', 'TEAM_ABBREVIATION_HOME', 
                                                'TEAM_ABBREVIATION_AWAY', 'PTS_AWAY', 'PTS_AWAY'], axis=1)
    
    to_return = {}
    to_return['df'] = df
    to_return['officials'] = officials
    to_return['inactive_players'] = inactive_players
    return to_return

# get proxies
proxies = [str(proxy).split('\\')[0][2:] for proxy in urllib.request.urlopen("https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=1500&country=all&ssl=yes&anonymity=all&simplified=true").readlines()]

# get common player info for each player in the db
dfs = []
game_ids = pd.read_sql('SELECT GAME_ID FROM Game', conn)['GAME_ID'].values
dfs = [get_box_score_summaries(game_id, proxies=proxies) for game_id in game_ids]
df = pd.concat([df['df'] for df in dfs if df != None], axis=0)
display(df)
officials = pd.concat([df['officials'] for df in dfs if df != None], axis=0)
display(officials)
inactive_players = pd.concat([df['inactive_players'] for df in dfs if df != None], axis = 0)
display(inactive_players)

In [ ]:
df.info(max_cols=150)

In [ ]:
officials.info()

In [ ]:
inactive_players.info()

In [ ]:
current_game_df = pd.read_sql('SELECT * FROM Game', conn)
current_game_df

In [ ]:
dff = df.copy()
dff = dff.loc[:,~dff.columns.duplicated()]
dfff = current_game_df.merge(dff, on=['GAME_ID']).dropna(axis=1, how='all')
dfff

In [ ]:
try:
    dfff.to_sql('Game', conn, index=False)
    officials.to_sql('Game_Officials', conn, index=False)
    inactive_players.to_sql('Game_Inactive_Players', conn, index=False)
except:
    pass

#### Upload to Kaggle

In [ ]:
!kaggle datasets version -p ../data -m "fixing line scores in Game table"

### All Time Leaders Grids

In [ ]:
dfs = alltimeleadersgrids.AllTimeLeadersGrids(TopX=4500).get_data_frames()

In [14]:
pipeline_content = "\
name: Basketball Dataset Pipeline - Montly Update # Pipeline name\
steps:\
  # retrieve queries to update teams & players lists (if needed)\
  - kernel: wyattowalsh/basketball-dataset-update-teams\
  - kernel: wyattowalsh/basketball-dataset-update-players\
  - status: 1s\
  # write queries to database\
  - output: wyattowalsh/basketball-dataset-update-teams\
  - output: wyattowalsh/basketball-dataset-update-players\
    "
pipeline_content

'name: Basketball Dataset Pipeline - Montly Update # Pipeline namesteps:  # retrieve queries to update teams & players lists (if needed)  - kernel: wyattowalsh/basketball-dataset-update-teams  - kernel: wyattowalsh/basketball-dataset-update-players  - status: 1s  # write queries to database  - output: wyattowalsh/basketball-dataset-update-teams  - output: wyattowalsh/basketball-dataset-update-players    '

In [ ]:
pd.read_html("https://www.ycombinator.com/topcompanies")[0]['CompanyOverview'].values

In [ ]:
dfs[2].info()

In [ ]:
dfs[3]

In [ ]:
dfs[3].info()

In [ ]:
dfs[4]

In [ ]:
dfs[4].info()

In [ ]:
dfs[5]

In [ ]:
dfs[5].info()

In [ ]:
dfs[6]

In [ ]:
dfs[6].info()

In [ ]:
dfs[7]

In [ ]:
dfs[7].info()

In [ ]:
dfs[8]

In [ ]:
dfs[8].info()

In [45]:
current_ids = pd.read_sql("SELECT {} FROM {}".format('GAME_ID', 'Game'), conn).T.values.flatten()
ex_row = pd.read_sql("SELECT * FROM {} WHERE {} = '{}'".format('Game', "GAME_ID", current_ids[0]), conn)
# sql = "INSERT INTO {} {} VALUE {}".format(table, "(" + ", ".join(db_row.columns.values) + ")", "(" + ", ".join(db_row.values) + ")")

ex_row

,GAME_ID,SEASON_ID,TEAM_ID_HOME,TEAM_ABBREVIATION_HOME,TEAM_NAME_HOME,GAME_DATE_HOME,MATCHUP_HOME,WL_HOME,MIN_HOME,FGM_HOME,FGA_HOME,FG_PCT_HOME,FG3M_HOME,FG3A_HOME,FG3_PCT_HOME,FTM_HOME,FTA_HOME,FT_PCT_HOME,OREB_HOME,DREB_HOME,REB_HOME,AST_HOME,STL_HOME,BLK_HOME,TOV_HOME,PF_HOME,PTS_HOME_x,PLUS_MINUS_HOME,VIDEO_AVAILABLE_HOME,TEAM_ID_AWAY,TEAM_ABBREVIATION_AWAY,TEAM_NAME_AWAY,GAME_DATE_AWAY,MATCHUP_AWAY,WL_AWAY,MIN_AWAY,FGM_AWAY,FGA_AWAY,FG_PCT_AWAY,FG3M_AWAY,FG3A_AWAY,FG3_PCT_AWAY,FTM_AWAY,FTA_AWAY,FT_PCT_AWAY,OREB_AWAY,DREB_AWAY,REB_AWAY,AST_AWAY,STL_AWAY,BLK_AWAY,TOV_AWAY,PF_AWAY,PTS_AWAY,PLUS_MINUS_AWAY,VIDEO_AVAILABLE_AWAY,GAME_DATE_EST,GAME_SEQUENCE,GAME_STATUS_ID,GAME_STATUS_TEXT,GAMECODE,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,LIVE_PERIOD,LIVE_PC_TIME,NATL_TV_BROADCASTER_ABBREVIATION,LIVE_PERIOD_TIME_BCAST,WH_STATUS,TEAM_CITY_HOME,PTS_PAINT_HOME,PTS_2ND_CHANCE_HOME,PTS_FB_HOME,LARGEST_LEAD_HOME,LEAD_CHANGES_HOME,TIMES_TIED_HOME,TEAM_TURNOVERS_HOME,TOTAL_TURNOVERS_HOME,TEAM_REBOUNDS_HOME,PTS_OFF_TO_HOME,TEAM_CITY_AWAY,PTS_PAINT_AWAY,PTS_2ND_CHANCE_AWAY,PTS_FB_AWAY,LARGEST_LEAD_AWAY,LEAD_CHANGES_AWAY,TIMES_TIED_AWAY,TEAM_TURNOVERS_AWAY,TOTAL_TURNOVERS_AWAY,TEAM_REBOUNDS_AWAY,PTS_OFF_TO_AWAY,LEAGUE_ID,GAME_DATE,ATTENDANCE,GAME_TIME,TEAM_CITY_NAME_HOME,TEAM_NICKNAME_HOME,TEAM_WINS_LOSSES_HOME,PTS_QTR1_HOME,PTS_QTR2_HOME,PTS_QTR3_HOME,PTS_QTR4_HOME,PTS_OT1_HOME,PTS_OT2_HOME,PTS_OT3_HOME,PTS_OT4_HOME,PTS_OT5_HOME,PTS_OT6_HOME,PTS_OT7_HOME,PTS_OT8_HOME,PTS_OT9_HOME,PTS_OT10_HOME,PTS_HOME_y,TEAM_CITY_NAME_AWAY,TEAM_NICKNAME_AWAY,TEAM_WINS_LOSSES_AWAY,PTS_QTR1_AWAY,PTS_QTR2_AWAY,PTS_QTR3_AWAY,PTS_QTR4_AWAY,PTS_OT1_AWAY,PTS_OT2_AWAY,PTS_OT3_AWAY,PTS_OT4_AWAY,PTS_OT5_AWAY,PTS_OT6_AWAY,PTS_OT7_AWAY,PTS_OT8_AWAY,PTS_OT9_AWAY,PTS_OT10_AWAY,LAST_GAME_ID,LAST_GAME_DATE_EST,LAST_GAME_HOME_TEAM_ID,LAST_GAME_HOME_TEAM_CITY,LAST_GAME_HOME_TEAM_NAME,LAST_GAME_HOME_TEAM_ABBREVIATION,LAST_GAME_HOME_TEAM_POINTS,LAST_GAME_VISITOR_TEAM_ID,LAST_GAME_VISITOR_TEAM_CITY,LAST_GAME_VISITOR_TEAM_NAME,LAST_GAME_VISITOR_TEAM_CITY1,LAST_GAME_VISITOR_TEAM_POINTS,HOME_TEAM_WINS,HOME_TEAM_LOSSES,SERIES_LEADER,VIDEO_AVAILABLE_FLAG,PT_AVAILABLE,PT_XYZ_AVAILABLE,HUSTLE_STATUS,HISTORICAL_STATUS
0,0024600001,21946,1610610035,HUS,Toronto Huskies,1946-11-01,HUS vs. NYK,L,0,25.0,None,None,None,None,None,16.0,29.0,0.552,None,None,None,None,None,None,None,None,66,-2,0,1610612752,NYK,New York Knicks,1946-11-01,NYK @ HUS,W,0,24.0,None,None,None,None,None,20.0,26.0,0.769,None,None,None,None,None,None,None,None,68,2,0,1946-11-01T00:00:00,None,3,,19461101/NYKHUS,1610610035,1610612752,1946,5.0,None,None,Q5 -,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"FRIDAY, NOVEMBER 1, 1946",None,,Toronto,Huskies,-,None,None,None,None,18,None,None,None,None,None,None,None,None,None,66.0,New York,Knicks,-,None,None,None,None,24,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,3.0,3.0,Tied,0.0,0.0,0.0,0.0,1.0


In [66]:
!kaggle datasets download -d wyattowalsh/basketball

100%|██████████████████████████████████████| 11.8M/11.8M [00:00<00:00, 15.6MB/s]
100%|██████████████████████████████████████| 11.8M/11.8M [00:00<00:00, 14.3MB/s]


In [25]:
import sqlite3 as sql
con = sql.connect('data/basketball.sqlite')

In [3]:
%load_ext sql
%sql sqlite:///data/basketball.sqlite

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @data/basketball.sqlite'

In [4]:
%%sql
SELECT COUNT(*) FROM Game

   sqlite:///basketball.sqlite
 * sqlite:///data/basketball.sqlite
Done.


COUNT(*)
62015


In [14]:
%%sql
CREATE TABLE temp_table as SELECT DISTINCT * FROM Game;
DELETE FROM Game; 
INSERT INTO Game SELECT * FROM temp_table
DROP table temp_table

 * sqlite:///basketball.sqlite
Done.
62181 rows affected.
62015 rows affected.


ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [18]:
%%sql
SELECT COUNT(*) FROM temp_table

 * sqlite:///basketball.sqlite
(sqlite3.OperationalError) no such table: temp_table
[SQL: SELECT COUNT(*) FROM temp_table]
(Background on this error at: http://sqlalche.me/e/14/e3q8)


In [ ]:
%%sql
temp_table

In [ ]:
%%sql
SELECT COUNT(*) FROM temp_table

In [19]:
%%sql
SELECT COUNT(*) FROM Game

 * sqlite:///basketball.sqlite
Done.


COUNT(*)
62015


In [21]:
%%sql
SELECT * FROM Game LIMIT 3

 * sqlite:///basketball.sqlite
Done.


GAME_ID,SEASON_ID,TEAM_ID_HOME,TEAM_ABBREVIATION_HOME,TEAM_NAME_HOME,GAME_DATE_HOME,MATCHUP_HOME,WL_HOME,MIN_HOME,FGM_HOME,FGA_HOME,FG_PCT_HOME,FG3M_HOME,FG3A_HOME,FG3_PCT_HOME,FTM_HOME,FTA_HOME,FT_PCT_HOME,OREB_HOME,DREB_HOME,REB_HOME,AST_HOME,STL_HOME,BLK_HOME,TOV_HOME,PF_HOME,PTS_HOME_x,PLUS_MINUS_HOME,VIDEO_AVAILABLE_HOME,TEAM_ID_AWAY,TEAM_ABBREVIATION_AWAY,TEAM_NAME_AWAY,GAME_DATE_AWAY,MATCHUP_AWAY,WL_AWAY,MIN_AWAY,FGM_AWAY,FGA_AWAY,FG_PCT_AWAY,FG3M_AWAY,FG3A_AWAY,FG3_PCT_AWAY,FTM_AWAY,FTA_AWAY,FT_PCT_AWAY,OREB_AWAY,DREB_AWAY,REB_AWAY,AST_AWAY,STL_AWAY,BLK_AWAY,TOV_AWAY,PF_AWAY,PTS_AWAY,PLUS_MINUS_AWAY,VIDEO_AVAILABLE_AWAY,GAME_DATE_EST,GAME_SEQUENCE,GAME_STATUS_ID,GAME_STATUS_TEXT,GAMECODE,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,LIVE_PERIOD,LIVE_PC_TIME,NATL_TV_BROADCASTER_ABBREVIATION,LIVE_PERIOD_TIME_BCAST,WH_STATUS,TEAM_CITY_HOME,PTS_PAINT_HOME,PTS_2ND_CHANCE_HOME,PTS_FB_HOME,LARGEST_LEAD_HOME,LEAD_CHANGES_HOME,TIMES_TIED_HOME,TEAM_TURNOVERS_HOME,TOTAL_TURNOVERS_HOME,TEAM_REBOUNDS_HOME,PTS_OFF_TO_HOME,TEAM_CITY_AWAY,PTS_PAINT_AWAY,PTS_2ND_CHANCE_AWAY,PTS_FB_AWAY,LARGEST_LEAD_AWAY,LEAD_CHANGES_AWAY,TIMES_TIED_AWAY,TEAM_TURNOVERS_AWAY,TOTAL_TURNOVERS_AWAY,TEAM_REBOUNDS_AWAY,PTS_OFF_TO_AWAY,LEAGUE_ID,GAME_DATE,ATTENDANCE,GAME_TIME,TEAM_CITY_NAME_HOME,TEAM_NICKNAME_HOME,TEAM_WINS_LOSSES_HOME,PTS_QTR1_HOME,PTS_QTR2_HOME,PTS_QTR3_HOME,PTS_QTR4_HOME,PTS_OT1_HOME,PTS_OT2_HOME,PTS_OT3_HOME,PTS_OT4_HOME,PTS_OT5_HOME,PTS_OT6_HOME,PTS_OT7_HOME,PTS_OT8_HOME,PTS_OT9_HOME,PTS_OT10_HOME,PTS_HOME_y,TEAM_CITY_NAME_AWAY,TEAM_NICKNAME_AWAY,TEAM_WINS_LOSSES_AWAY,PTS_QTR1_AWAY,PTS_QTR2_AWAY,PTS_QTR3_AWAY,PTS_QTR4_AWAY,PTS_OT1_AWAY,PTS_OT2_AWAY,PTS_OT3_AWAY,PTS_OT4_AWAY,PTS_OT5_AWAY,PTS_OT6_AWAY,PTS_OT7_AWAY,PTS_OT8_AWAY,PTS_OT9_AWAY,PTS_OT10_AWAY,LAST_GAME_ID,LAST_GAME_DATE_EST,LAST_GAME_HOME_TEAM_ID,LAST_GAME_HOME_TEAM_CITY,LAST_GAME_HOME_TEAM_NAME,LAST_GAME_HOME_TEAM_ABBREVIATION,LAST_GAME_HOME_TEAM_POINTS,LAST_GAME_VISITOR_TEAM_ID,LAST_GAME_VISITOR_TEAM_CITY,LAST_GAME_VISITOR_TEAM_NAME,LAST_GAME_VISITOR_TEAM_CITY1,LAST_GAME_VISITOR_TEAM_POINTS,HOME_TEAM_WINS,HOME_TEAM_LOSSES,SERIES_LEADER,VIDEO_AVAILABLE_FLAG,PT_AVAILABLE,PT_XYZ_AVAILABLE,HUSTLE_STATUS,HISTORICAL_STATUS
0024600001,21946,1610610035,HUS,Toronto Huskies,1946-11-01,HUS vs. NYK,L,0,25.0,None,None,None,None,None,16.0,29.0,0.552,None,None,None,None,None,None,None,None,66,-2,0,1610612752,NYK,New York Knicks,1946-11-01,NYK @ HUS,W,0,24.0,None,None,None,None,None,20.0,26.0,0.769,None,None,None,None,None,None,None,None,68,2,0,1946-11-01T00:00:00,None,3,,19461101/NYKHUS,1610610035,1610612752,1946,5.0,None,None,Q5 -,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"FRIDAY, NOVEMBER 1, 1946",None,,Toronto,Huskies,-,None,None,None,None,18,None,None,None,None,None,None,None,None,None,66.0,New York,Knicks,-,None,None,None,None,24,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,3.0,3.0,Tied,0.0,0.0,0.0,0.0,1.0
0024600003,21946,1610610034,BOM,St. Louis Bombers,1946-11-02,BOM vs. PIT,W,0,20.0,59.0,0.339,None,None,None,16.0,None,None,None,None,None,None,None,None,None,21.0,56,5,0,1610610031,PIT,Pittsburgh Ironmen,1946-11-02,PIT @ BOM,L,0,16.0,72.0,0.222,None,None,None,19.0,None,None,None,None,None,None,None,None,None,25.0,51,-5,0,1946-11-02T00:00:00,None,3,,19461102/PITBOM,1610610034,1610610031,1946,4.0,None,None,Q4 -,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"SATURDAY, NOVEMBER 2, 1946",None,,St. Louis,Bombers,-,16,16,18,6,None,None,None,None,None,None,None,None,None,None,56.0,Pittsburgh,Ironmen,-,5,15,17,14,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,5.0,1.0,St. Louis,0.0,0.0,0.0,0.0,1.0
0024600004,21946,1610610025,CHS,Chicago Stags,1946-11-02,CHS vs. NYK,W,0,21.0,None,None,None,None,None,21.0,None,None,None,None,None,None,None

In [22]:
%%sql
ALTER TABLE Game
RENAME COLUMN PTS_HOME_x TO PTS_HOME

 * sqlite:///basketball.sqlite
Done.


1


In [23]:
%%sql
SELECT * FROM Game LIMIT 1

 * sqlite:///basketball.sqlite
Done.


GAME_ID,SEASON_ID,TEAM_ID_HOME,TEAM_ABBREVIATION_HOME,TEAM_NAME_HOME,GAME_DATE_HOME,MATCHUP_HOME,WL_HOME,MIN_HOME,FGM_HOME,FGA_HOME,FG_PCT_HOME,FG3M_HOME,FG3A_HOME,FG3_PCT_HOME,FTM_HOME,FTA_HOME,FT_PCT_HOME,OREB_HOME,DREB_HOME,REB_HOME,AST_HOME,STL_HOME,BLK_HOME,TOV_HOME,PF_HOME,PTS_HOME,PLUS_MINUS_HOME,VIDEO_AVAILABLE_HOME,TEAM_ID_AWAY,TEAM_ABBREVIATION_AWAY,TEAM_NAME_AWAY,GAME_DATE_AWAY,MATCHUP_AWAY,WL_AWAY,MIN_AWAY,FGM_AWAY,FGA_AWAY,FG_PCT_AWAY,FG3M_AWAY,FG3A_AWAY,FG3_PCT_AWAY,FTM_AWAY,FTA_AWAY,FT_PCT_AWAY,OREB_AWAY,DREB_AWAY,REB_AWAY,AST_AWAY,STL_AWAY,BLK_AWAY,TOV_AWAY,PF_AWAY,PTS_AWAY,PLUS_MINUS_AWAY,VIDEO_AVAILABLE_AWAY,GAME_DATE_EST,GAME_SEQUENCE,GAME_STATUS_ID,GAME_STATUS_TEXT,GAMECODE,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,LIVE_PERIOD,LIVE_PC_TIME,NATL_TV_BROADCASTER_ABBREVIATION,LIVE_PERIOD_TIME_BCAST,WH_STATUS,TEAM_CITY_HOME,PTS_PAINT_HOME,PTS_2ND_CHANCE_HOME,PTS_FB_HOME,LARGEST_LEAD_HOME,LEAD_CHANGES_HOME,TIMES_TIED_HOME,TEAM_TURNOVERS_HOME,TOTAL_TURNOVERS_HOME,TEAM_REBOUNDS_HOME,PTS_OFF_TO_HOME,TEAM_CITY_AWAY,PTS_PAINT_AWAY,PTS_2ND_CHANCE_AWAY,PTS_FB_AWAY,LARGEST_LEAD_AWAY,LEAD_CHANGES_AWAY,TIMES_TIED_AWAY,TEAM_TURNOVERS_AWAY,TOTAL_TURNOVERS_AWAY,TEAM_REBOUNDS_AWAY,PTS_OFF_TO_AWAY,LEAGUE_ID,GAME_DATE,ATTENDANCE,GAME_TIME,TEAM_CITY_NAME_HOME,TEAM_NICKNAME_HOME,TEAM_WINS_LOSSES_HOME,PTS_QTR1_HOME,PTS_QTR2_HOME,PTS_QTR3_HOME,PTS_QTR4_HOME,PTS_OT1_HOME,PTS_OT2_HOME,PTS_OT3_HOME,PTS_OT4_HOME,PTS_OT5_HOME,PTS_OT6_HOME,PTS_OT7_HOME,PTS_OT8_HOME,PTS_OT9_HOME,PTS_OT10_HOME,PTS_HOME_y,TEAM_CITY_NAME_AWAY,TEAM_NICKNAME_AWAY,TEAM_WINS_LOSSES_AWAY,PTS_QTR1_AWAY,PTS_QTR2_AWAY,PTS_QTR3_AWAY,PTS_QTR4_AWAY,PTS_OT1_AWAY,PTS_OT2_AWAY,PTS_OT3_AWAY,PTS_OT4_AWAY,PTS_OT5_AWAY,PTS_OT6_AWAY,PTS_OT7_AWAY,PTS_OT8_AWAY,PTS_OT9_AWAY,PTS_OT10_AWAY,LAST_GAME_ID,LAST_GAME_DATE_EST,LAST_GAME_HOME_TEAM_ID,LAST_GAME_HOME_TEAM_CITY,LAST_GAME_HOME_TEAM_NAME,LAST_GAME_HOME_TEAM_ABBREVIATION,LAST_GAME_HOME_TEAM_POINTS,LAST_GAME_VISITOR_TEAM_ID,LAST_GAME_VISITOR_TEAM_CITY,LAST_GAME_VISITOR_TEAM_NAME,LAST_GAME_VISITOR_TEAM_CITY1,LAST_GAME_VISITOR_TEAM_POINTS,HOME_TEAM_WINS,HOME_TEAM_LOSSES,SERIES_LEADER,VIDEO_AVAILABLE_FLAG,PT_AVAILABLE,PT_XYZ_AVAILABLE,HUSTLE_STATUS,HISTORICAL_STATUS
0024600001,21946,1610610035,HUS,Toronto Huskies,1946-11-01,HUS vs. NYK,L,0,25.0,None,None,None,None,None,16.0,29.0,0.552,None,None,None,None,None,None,None,None,66,-2,0,1610612752,NYK,New York Knicks,1946-11-01,NYK @ HUS,W,0,24.0,None,None,None,None,None,20.0,26.0,0.769,None,None,None,None,None,None,None,None,68,2,0,1946-11-01T00:00:00,None,3,,19461101/NYKHUS,1610610035,1610612752,1946,5.0,None,None,Q5 -,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"FRIDAY, NOVEMBER 1, 1946",None,,Toronto,Huskies,-,None,None,None,None,18,None,None,None,None,None,None,None,None,None,66.0,New York,Knicks,-,None,None,None,None,24,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,3.0,3.0,Tied,0.0,0.0,0.0,0.0,1.0


In [18]:
import subprocess

In [26]:
subprocess.run("mkdir ./data/", shell=True)
subprocess.run("mv ./basketball.sqlite ./data/", shell=True)
with open('./data/dataset-metadata.json', 'w') as fp:
    json.dump({"id": "wyattowalsh/basketball"}, fp)
subprocess.run("kaggle datasets version -p ./data -m '{}'".format('cleaned up column names and removed duplicate rows'), shell=True)

CompletedProcess(args="kaggle datasets version -p ./data -m 'cleaned up column names and removed duplicate rows'", returncode=0)

In [5]:
%%sql result_set << 
SELECT * FROM Game LIMIT 2

   sqlite:///basketball.sqlite
 * sqlite:///data/basketball.sqlite
Done.
Returning data to local variable result_set


In [9]:
df = result_set.DataFrame()

In [10]:
df.loc[df['GAME_ID'] == '0024600001']['GAME_ID'].values[0]

'0024600001'

In [11]:
%%sql
SELECT * FROM GAME ORDER BY GAME_ID DESC LIMIT 1

   sqlite:///basketball.sqlite
 * sqlite:///data/basketball.sqlite
Done.


GAME_ID,SEASON_ID,TEAM_ID_HOME,TEAM_ABBREVIATION_HOME,TEAM_NAME_HOME,GAME_DATE_HOME,MATCHUP_HOME,WL_HOME,MIN_HOME,FGM_HOME,FGA_HOME,FG_PCT_HOME,FG3M_HOME,FG3A_HOME,FG3_PCT_HOME,FTM_HOME,FTA_HOME,FT_PCT_HOME,OREB_HOME,DREB_HOME,REB_HOME,AST_HOME,STL_HOME,BLK_HOME,TOV_HOME,PF_HOME,PTS_HOME,PLUS_MINUS_HOME,VIDEO_AVAILABLE_HOME,TEAM_ID_AWAY,TEAM_ABBREVIATION_AWAY,TEAM_NAME_AWAY,GAME_DATE_AWAY,MATCHUP_AWAY,WL_AWAY,MIN_AWAY,FGM_AWAY,FGA_AWAY,FG_PCT_AWAY,FG3M_AWAY,FG3A_AWAY,FG3_PCT_AWAY,FTM_AWAY,FTA_AWAY,FT_PCT_AWAY,OREB_AWAY,DREB_AWAY,REB_AWAY,AST_AWAY,STL_AWAY,BLK_AWAY,TOV_AWAY,PF_AWAY,PTS_AWAY,PLUS_MINUS_AWAY,VIDEO_AVAILABLE_AWAY,GAME_DATE_EST,GAME_SEQUENCE,GAME_STATUS_ID,GAME_STATUS_TEXT,GAMECODE,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,LIVE_PERIOD,LIVE_PC_TIME,NATL_TV_BROADCASTER_ABBREVIATION,LIVE_PERIOD_TIME_BCAST,WH_STATUS,TEAM_CITY_HOME,PTS_PAINT_HOME,PTS_2ND_CHANCE_HOME,PTS_FB_HOME,LARGEST_LEAD_HOME,LEAD_CHANGES_HOME,TIMES_TIED_HOME,TEAM_TURNOVERS_HOME,TOTAL_TURNOVERS_HOME,TEAM_REBOUNDS_HOME,PTS_OFF_TO_HOME,TEAM_CITY_AWAY,PTS_PAINT_AWAY,PTS_2ND_CHANCE_AWAY,PTS_FB_AWAY,LARGEST_LEAD_AWAY,LEAD_CHANGES_AWAY,TIMES_TIED_AWAY,TEAM_TURNOVERS_AWAY,TOTAL_TURNOVERS_AWAY,TEAM_REBOUNDS_AWAY,PTS_OFF_TO_AWAY,LEAGUE_ID,GAME_DATE,ATTENDANCE,GAME_TIME,TEAM_CITY_NAME_HOME,TEAM_NICKNAME_HOME,TEAM_WINS_LOSSES_HOME,PTS_QTR1_HOME,PTS_QTR2_HOME,PTS_QTR3_HOME,PTS_QTR4_HOME,PTS_OT1_HOME,PTS_OT2_HOME,PTS_OT3_HOME,PTS_OT4_HOME,PTS_OT5_HOME,PTS_OT6_HOME,PTS_OT7_HOME,PTS_OT8_HOME,PTS_OT9_HOME,PTS_OT10_HOME,PTS_HOME_y,TEAM_CITY_NAME_AWAY,TEAM_NICKNAME_AWAY,TEAM_WINS_LOSSES_AWAY,PTS_QTR1_AWAY,PTS_QTR2_AWAY,PTS_QTR3_AWAY,PTS_QTR4_AWAY,PTS_OT1_AWAY,PTS_OT2_AWAY,PTS_OT3_AWAY,PTS_OT4_AWAY,PTS_OT5_AWAY,PTS_OT6_AWAY,PTS_OT7_AWAY,PTS_OT8_AWAY,PTS_OT9_AWAY,PTS_OT10_AWAY,LAST_GAME_ID,LAST_GAME_DATE_EST,LAST_GAME_HOME_TEAM_ID,LAST_GAME_HOME_TEAM_CITY,LAST_GAME_HOME_TEAM_NAME,LAST_GAME_HOME_TEAM_ABBREVIATION,LAST_GAME_HOME_TEAM_POINTS,LAST_GAME_VISITOR_TEAM_ID,LAST_GAME_VISITOR_TEAM_CITY,LAST_GAME_VISITOR_TEAM_NAME,LAST_GAME_VISITOR_TEAM_CITY1,LAST_GAME_VISITOR_TEAM_POINTS,HOME_TEAM_WINS,HOME_TEAM_LOSSES,SERIES_LEADER,VIDEO_AVAILABLE_FLAG,PT_AVAILABLE,PT_XYZ_AVAILABLE,HUSTLE_STATUS,HISTORICAL_STATUS
0029901189,21999,1610612760,SEA,Seattle SuperSonics,2000-04-19,SEA vs. LAC,L,240,44.0,87,0.506,8,21,0.381,10.0,19.0,0.526,13,22,35,27,7,7,12,21.0,106,-8,0,1610612746,LAC,Los Angeles Clippers,2000-04-19,LAC @ SEA,W,240,41.0,94,0.436,12,19,0.632,20.0,24.0,0.833,20,29,49,25,5,5,12,16.0,114,8,0,2000-04-19T00:00:00,14,3,Final,20000419/LACSEA,1610612760,1610612746,1999,4.0,,None,Q4 -,1.0,Seattle,48,17,23,9,7,3,0,12,11,0,Los Angeles,28,15,10,11,7,3,1,12,14,0,00,"WEDNESDAY, APRIL 19, 2000",15373,2:09,Seattle,SuperSonics,-,36,26,19,25,0,0,0,0,None,None,None,None,None,None,106.0,Los Angeles,Clippers,-,34,20,28,32,0,0,0,0,None,None,None,None,None,None,0029900239,1999-12-04T00:00:00,1610612760,Seattle,SuperSonics,SEA,102,1610612746,Los Angeles,Clippers,LAC,89,3.0,1.0,Seattle,0.0,0.0,0.0,0.0,0.0


In [14]:
%%sql
ALTER TABLE Game
RENAME COLUMN GAME_DATE TO GAME_DATE_DAY

   sqlite:///basketball.sqlite
 * sqlite:///data/basketball.sqlite
Done.


1


In [15]:
%%sql
ALTER TABLE Game
RENAME COLUMN GAME_DATE_HOME TO GAME_DATE

   sqlite:///basketball.sqlite
 * sqlite:///data/basketball.sqlite
Done.


1


In [16]:
%%sql
SELECT * FROM GAME ORDER BY GAME_ID DESC LIMIT 1

   sqlite:///basketball.sqlite
 * sqlite:///data/basketball.sqlite
Done.


GAME_ID,SEASON_ID,TEAM_ID_HOME,TEAM_ABBREVIATION_HOME,TEAM_NAME_HOME,GAME_DATE,MATCHUP_HOME,WL_HOME,MIN_HOME,FGM_HOME,FGA_HOME,FG_PCT_HOME,FG3M_HOME,FG3A_HOME,FG3_PCT_HOME,FTM_HOME,FTA_HOME,FT_PCT_HOME,OREB_HOME,DREB_HOME,REB_HOME,AST_HOME,STL_HOME,BLK_HOME,TOV_HOME,PF_HOME,PTS_HOME,PLUS_MINUS_HOME,VIDEO_AVAILABLE_HOME,TEAM_ID_AWAY,TEAM_ABBREVIATION_AWAY,TEAM_NAME_AWAY,GAME_DATE_AWAY,MATCHUP_AWAY,WL_AWAY,MIN_AWAY,FGM_AWAY,FGA_AWAY,FG_PCT_AWAY,FG3M_AWAY,FG3A_AWAY,FG3_PCT_AWAY,FTM_AWAY,FTA_AWAY,FT_PCT_AWAY,OREB_AWAY,DREB_AWAY,REB_AWAY,AST_AWAY,STL_AWAY,BLK_AWAY,TOV_AWAY,PF_AWAY,PTS_AWAY,PLUS_MINUS_AWAY,VIDEO_AVAILABLE_AWAY,GAME_DATE_EST,GAME_SEQUENCE,GAME_STATUS_ID,GAME_STATUS_TEXT,GAMECODE,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,LIVE_PERIOD,LIVE_PC_TIME,NATL_TV_BROADCASTER_ABBREVIATION,LIVE_PERIOD_TIME_BCAST,WH_STATUS,TEAM_CITY_HOME,PTS_PAINT_HOME,PTS_2ND_CHANCE_HOME,PTS_FB_HOME,LARGEST_LEAD_HOME,LEAD_CHANGES_HOME,TIMES_TIED_HOME,TEAM_TURNOVERS_HOME,TOTAL_TURNOVERS_HOME,TEAM_REBOUNDS_HOME,PTS_OFF_TO_HOME,TEAM_CITY_AWAY,PTS_PAINT_AWAY,PTS_2ND_CHANCE_AWAY,PTS_FB_AWAY,LARGEST_LEAD_AWAY,LEAD_CHANGES_AWAY,TIMES_TIED_AWAY,TEAM_TURNOVERS_AWAY,TOTAL_TURNOVERS_AWAY,TEAM_REBOUNDS_AWAY,PTS_OFF_TO_AWAY,LEAGUE_ID,GAME_DATE_DAY,ATTENDANCE,GAME_TIME,TEAM_CITY_NAME_HOME,TEAM_NICKNAME_HOME,TEAM_WINS_LOSSES_HOME,PTS_QTR1_HOME,PTS_QTR2_HOME,PTS_QTR3_HOME,PTS_QTR4_HOME,PTS_OT1_HOME,PTS_OT2_HOME,PTS_OT3_HOME,PTS_OT4_HOME,PTS_OT5_HOME,PTS_OT6_HOME,PTS_OT7_HOME,PTS_OT8_HOME,PTS_OT9_HOME,PTS_OT10_HOME,PTS_HOME_y,TEAM_CITY_NAME_AWAY,TEAM_NICKNAME_AWAY,TEAM_WINS_LOSSES_AWAY,PTS_QTR1_AWAY,PTS_QTR2_AWAY,PTS_QTR3_AWAY,PTS_QTR4_AWAY,PTS_OT1_AWAY,PTS_OT2_AWAY,PTS_OT3_AWAY,PTS_OT4_AWAY,PTS_OT5_AWAY,PTS_OT6_AWAY,PTS_OT7_AWAY,PTS_OT8_AWAY,PTS_OT9_AWAY,PTS_OT10_AWAY,LAST_GAME_ID,LAST_GAME_DATE_EST,LAST_GAME_HOME_TEAM_ID,LAST_GAME_HOME_TEAM_CITY,LAST_GAME_HOME_TEAM_NAME,LAST_GAME_HOME_TEAM_ABBREVIATION,LAST_GAME_HOME_TEAM_POINTS,LAST_GAME_VISITOR_TEAM_ID,LAST_GAME_VISITOR_TEAM_CITY,LAST_GAME_VISITOR_TEAM_NAME,LAST_GAME_VISITOR_TEAM_CITY1,LAST_GAME_VISITOR_TEAM_POINTS,HOME_TEAM_WINS,HOME_TEAM_LOSSES,SERIES_LEADER,VIDEO_AVAILABLE_FLAG,PT_AVAILABLE,PT_XYZ_AVAILABLE,HUSTLE_STATUS,HISTORICAL_STATUS
0029901189,21999,1610612760,SEA,Seattle SuperSonics,2000-04-19,SEA vs. LAC,L,240,44.0,87,0.506,8,21,0.381,10.0,19.0,0.526,13,22,35,27,7,7,12,21.0,106,-8,0,1610612746,LAC,Los Angeles Clippers,2000-04-19,LAC @ SEA,W,240,41.0,94,0.436,12,19,0.632,20.0,24.0,0.833,20,29,49,25,5,5,12,16.0,114,8,0,2000-04-19T00:00:00,14,3,Final,20000419/LACSEA,1610612760,1610612746,1999,4.0,,None,Q4 -,1.0,Seattle,48,17,23,9,7,3,0,12,11,0,Los Angeles,28,15,10,11,7,3,1,12,14,0,00,"WEDNESDAY, APRIL 19, 2000",15373,2:09,Seattle,SuperSonics,-,36,26,19,25,0,0,0,0,None,None,None,None,None,None,106.0,Los Angeles,Clippers,-,34,20,28,32,0,0,0,0,None,None,None,None,None,None,0029900239,1999-12-04T00:00:00,1610612760,Seattle,SuperSonics,SEA,102,1610612746,Los Angeles,Clippers,LAC,89,3.0,1.0,Seattle,0.0,0.0,0.0,0.0,0.0


In [19]:
subprocess.run("kaggle datasets version -p ./data -m '{}'".format('Changed game date column names in Game table'), shell=True)

CompletedProcess(args="kaggle datasets version -p ./data -m 'Changed game date column names in Game table'", returncode=0)

In [20]:
row=df.loc[df['GAME_ID'] == '0024600001']
row

,GAME_ID,SEASON_ID,TEAM_ID_HOME,TEAM_ABBREVIATION_HOME,TEAM_NAME_HOME,GAME_DATE_HOME,MATCHUP_HOME,WL_HOME,MIN_HOME,FGM_HOME,...,LAST_GAME_VISITOR_TEAM_CITY1,LAST_GAME_VISITOR_TEAM_POINTS,HOME_TEAM_WINS,HOME_TEAM_LOSSES,SERIES_LEADER,VIDEO_AVAILABLE_FLAG,PT_AVAILABLE,PT_XYZ_AVAILABLE,HUSTLE_STATUS,HISTORICAL_STATUS
0,0024600001,21946,1610610035,HUS,Toronto Huskies,1946-11-01,HUS vs. NYK,L,0,25.0,...,None,None,3.0,3.0,Tied,0.0,0.0,0.0,0.0,1.0


In [27]:
import pandas as pd
a = pd.read_sql("SELECT {} FROM {}".format('GAME_ID', 'GAME'), con).T.values.flatten()

In [28]:
for b in a:
    print(b)

0024600001
0024600003
0024600004
0024600002
0024600005
0024600006
0024600007
0024600008
0024600009
0024600010
0024600012
0024600011
0024600013
0024600015
0024600016
0024600017
0024600014
0024600018
0024600020
0024600019
0024600022
0024600021
0024600023
0024600024
0024600025
0024600030
0024600028
0024600029
0024600027
0024600026
0024600031
0024600032
0024600033
0024600035
0024600034
0024600036
0024600038
0024600037
0024600041
0024600039
0024600040
0024600042
0024600045
0024600046
0024600044
0024600043
0024600048
0024600047
0024600049
0024600050
0024600052
0024600051
0024600053
0024600056
0024600054
0024600055
0024600059
0024600060
0024600057
0024600058
0024600061
0024600063
0024600062
0024600065
0024600064
0024600066
0024600067
0024600068
0024600069
0024600071
0024600070
0024600072
0024600074
0024600076
0024600075
0024600073
0024600078
0024600080
0024600079
0024600077
0024600081
0024600083
0024600082
0024600084
0024600085
0024600086
0024600088
0024600087
0024600089
0024600091
0024600092

0024800221
0024800222
0024800223
0024800224
0024800225
0024800229
0024800226
0024800230
0024800228
0024800227
0024800231
0024800232
0024800235
0024800233
0024800234
0024800238
0024800237
0024800236
0024800240
0024800241
0024800239
0024800242
0024800243
0024800244
0024800247
0024800246
0024800245
0024800248
0024800250
0024800251
0024800249
0024800252
0024800253
0024800254
0024800256
0024800255
0024800259
0024800258
0024800257
0024800262
0024800260
0024800261
0024800264
0024800263
0024800269
0024800268
0024800266
0024800265
0024800267
0024800270
0024800271
0024800273
0024800272
0024800275
0024800274
0024800276
0024800277
0024800279
0024800280
0024800278
0024800281
0024800282
0024800285
0024800286
0024800284
0024800283
0024800287
0024800289
0024800288
0024800291
0024800290
0024800292
0024800294
0024800293
0024800295
0024800297
0024800298
0024800296
0024800299
0024800303
0024800304
0024800302
0024800301
0024800300
0024800305
0024800306
0024800307
0024800309
0024800308
0024800310
0024800311

0025000044
0025000041
0025000045
0025000047
0025000049
0025000048
0025000046
0025000054
0025000053
0025000050
0025000052
0025000051
0025000055
0025000057
0025000056
0025000058
0025000061
0025000059
0025000060
0025000063
0025000064
0025000065
0025000062
0025000066
0025000067
0025000068
0025000070
0025000069
0025000073
0025000072
0025000074
0025000071
0025000075
0025000076
0025000077
0025000079
0025000078
0025000081
0025000080
0025000084
0025000082
0025000083
0025000085
0025000088
0025000086
0025000087
0025000090
0025000091
0025000092
0025000093
0025000089
0025000094
0025000098
0025000096
0025000095
0025000097
0025000100
0025000101
0025000099
0025000102
0025000104
0025000103
0025000105
0025000108
0025000106
0025000110
0025000109
0025000107
0025000113
0025000111
0025000112
0025000114
0025000116
0025000117
0025000115
0025000118
0025000119
0025000120
0025000123
0025000122
0025000125
0025000121
0025000124
0025000128
0025000129
0025000126
0025000127
0025000131
0025000132
0025000130
0025000135

0025200105
0025200106
0025200107
0025200108
0025200110
0025200109
0025200112
0025200114
0025200111
0025200113
0025200115
0025200118
0025200117
0025200116
0025200119
0025200120
0025200121
0025200124
0025200125
0025200123
0025200122
0025200127
0025200126
0025200128
0025200129
0025200130
0025200131
0025200133
0025200135
0025200134
0025200132
0025200136
0025200137
0025200141
0025200138
0025200140
0025200139
0025200145
0025200142
0025200143
0025200144
0025200147
0025200146
0025200148
0025200151
0025200149
0025200150
0025200152
0025200005
0025200153
0025200154
0025200157
0025200156
0025200162
0025200161
0025200158
0025200159
0025200160
0025200163
0025200165
0025200166
0025200164
0025200167
0025200169
0025200168
0025200171
0025200170
0025200173
0025200174
0025200172
0025200175
0025200176
0025200179
0025200180
0025200178
0025200177
0025200181
0025200183
0025200185
0025200184
0025200182
0025200186
0025200187
0025200188
0025200189
0025200191
0025200190
0025200193
0025200192
0025200194
0025200197

0025400170
0025400173
0025400175
0025400172
0025400174
0025400176
0025400178
0025400179
0025400177
0025400182
0025400180
0025400181
0025400183
0025400184
0025400185
0025400186
0025400187
0025400190
0025400189
0025400188
0025400192
0025400191
0025400193
0025400195
0025400196
0025400194
0025400197
0025400198
0025400199
0025400200
0025400201
0025400203
0025400202
0025400204
0025400205
0025400206
0025400207
0025400208
0025400209
0025400210
0025400212
0025400213
0025400211
0025400214
0025400215
0025400216
0025400217
0025400218
0025400220
0025400219
0025400221
0025400222
0025400223
0025400225
0025400226
0025400224
0025400227
0025400229
0025400228
0025400230
0025400231
0025400232
0025400234
0025400233
0025400235
0025400236
0025400237
0025400238
0025400241
0025400240
0025400239
0025400242
0025400243
0025400244
0025400246
0025400245
0025400249
0025400247
0025400248
0025400253
0025400250
0025400252
0025400251
0025400254
0025400255
0025400256
0025400258
0025400257
0025400260
0025400259
0025400261

...

0020400645
0020400647
0020400646
0020400642
0020400650
0020400648
0020400652
0020400649
0020400651
0020400653
0020400654
0020400658
0020400657
0020400655
0020400656
0020400659
0020400666
0020400667
0020400660
0020400663
0020400661
0020400665
0020400664
0020400668
0020400662
0020400676
0020400670
0020400674
0020400669
0020400671
0020400675
0020400672
0020400673
0020400677
0020400678
0020400679
0020400685
0020400687
0020400680
0020400686
0020400688
0020400681
0020400683
0020400682
0020400684
0020400689
0020400698
0020400697
0020400691
0020400694
0020400696
0020400695
0020400693
0020400690
0020400692
0020400702
0020400699
0020400701
0020400700
0020400705
0020400704
0020400707
0020400706
0020400703
0020400715
0020400714
0020400713
0020400708
0020400710
0020400709
0020400712
0020400711
0020400716
0020400721
0020400723
0020400722
0020400725
0020400720
0020400724
0020400717
0020400726
0020400719
0020400718
0020400729
0020400728
0020400727
0020400733
0020400740
0020400730
0020400739
0020400732

0020500387
0020500396
0020500398
0020500402
0020500401
0020500403
0020500399
0020500400
0020500397
0020500404
0020500412
0020500411
0020500407
0020500410
0020500405
0020500409
0020500406
0020500413
0020500408
0020500416
0020500415
0020500414
0020500419
0020500420
0020500423
0020500425
0020500424
0020500422
0020500418
0020500417
0020500421
0020500429
0020500430
0020500427
0020500426
0020500433
0020500428
0020500432
0020500431
0020500436
0020500434
0020500435
0020500439
0020500869
0020500440
0020500438
0020500437
0020500446
0020500445
0020500447
0020500443
0020500442
0020500444
0020500441
0020500454
0020500455
0020500453
0020500450
0020500452
0020500448
0020500449
0020500456
0020500451
0020500457
0020500458
0020500461
0020500466
0020500468
0020500471
0020500469
0020500467
0020500463
0020500459
0020500464
0020500460
0020500465
0020500470
0020500462
0020500478
0020500477
0020500479
0020500474
0020500475
0020500476
0020500473
0020500480
0020500472
0020500483
0020500482
0020500485
0020500484

0020600105
0020600100
0020600103
0020600102
0020600114
0020600112
0020600111
0020600110
0020600108
0020600109
0020600113
0020600115
0020600116
0020600117
0020600118
0020600125
0020600127
0020600126
0020600120
0020600122
0020600121
0020600123
0020600119
0020600124
0020600128
0020600141
0020600133
0020600132
0020600140
0020600131
0020600134
0020600139
0020600136
0020600137
0020600130
0020600138
0020600135
0020600129
0020600142
0020600143
0020600150
0020600145
0020600148
0020600147
0020600146
0020600149
0020600144
0020600151
0020600157
0020600152
0020600153
0020600155
0020600156
0020600154
0020600170
0020600162
0020600166
0020600169
0020600160
0020600165
0020600161
0020600168
0020600163
0020600167
0020600164
0020600158
0020600159
0020600172
0020600179
0020600182
0020600171
0020600174
0020600175
0020600181
0020600176
0020600173
0020600180
0020600178
0020600177
0020600183
0020600194
0020600187
0020600186
0020600188
0020600190
0020600191
0020600193
0020600189
0020600184
0020600185
0020600192

0020700057
0020700056
0020700054
0020700061
0020700065
0020700064
0020700063
0020700066
0020700076
0020700069
0020700068
0020700074
0020700070
0020700072
0020700075
0020700077
0020700071
0020700073
0020700067
0020700078
0020700082
0020700081
0020700079
0020700085
0020700083
0020700080
0020700084
0020700088
0020700090
0020700087
0020700092
0020700086
0020700089
0020700091
0020700093
0020700095
0020700094
0020700101
0020700102
0020700097
0020700103
0020700098
0020700100
0020700099
0020700096
0020700116
0020700108
0020700115
0020700107
0020700113
0020700109
0020700110
0020700106
0020700114
0020700112
0020700111
0020700105
0020700104
0020700118
0020700117
0020700120
0020700130
0020700126
0020700129
0020700122
0020700121
0020700124
0020700123
0020700128
0020700127
0020700119
0020700125
0020700134
0020700140
0020700137
0020700136
0020700138
0020700133
0020700139
0020700135
0020700131
0020700132
0020700144
0020700142
0020700143
0020700141
0020700148
0020700147
0020700146
0020700145
0020700153

KeyboardInterrupt: 

In [32]:
df = leaguegamelog.LeagueGameLog(date_from_nullable='03-20-2021', timeout=3).get_data_frames()[0]

In [33]:
df['GAME_ID'].values

array(['0022000640', '0022000640', '0022000642', '0022000642',
       '0022000641', '0022000641', '0022000643', '0022000643',
       '0022000644', '0022000644', '0022000645', '0022000645',
       '0022000016', '0022000016', '0022000646', '0022000646',
       '0022000167', '0022000167', '0022000647', '0022000647',
       '0022000649', '0022000649', '0022000648', '0022000648',
       '0022000650', '0022000650', '0022000651', '0022000651',
       '0022000652', '0022000652', '0022000653', '0022000653',
       '0022000654', '0022000654', '0022000655', '0022000655',
       '0022000656', '0022000656', '0022000658', '0022000658',
       '0022000657', '0022000657', '0022000659', '0022000659',
       '0022000660', '0022000660', '0022000664', '0022000664',
       '0022000661', '0022000661', '0022000662', '0022000662',
       '0022000663', '0022000663', '0022000666', '0022000666',
       '0022000665', '0022000665', '0022000667', '0022000667',
       '0022000672', '0022000672', '0022000675', '00220

In [48]:
df.loc[(df['MATCHUP'] == 'ATL @ LAL')]['GAME_DATE'].values[0]

'2021-03-20'

In [42]:
df.loc[(df['GAME_ID'] == '0022000640')]['TEAM_ID'].values[0]

1610612737